<a href="https://colab.research.google.com/github/mmistroni/jupyter/blob/master/Form_10Q_Noteboook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas-datareader
import requests
import pandas as pd
from datetime import date
import logging
from pandas.tseries.offsets import BDay
import requests
import numpy as np
import urllib
import json
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as dr
import numpy as np
from datetime import datetime, date
import pandas as pd



In [ ]:
from google.colab import auth
auth.authenticate_user()

!mkdir -p data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

def get_iexapi_keys():
  with open('gdrive/My Drive/passwords/iexapi.keys') as f:
    return f.readlines()[0]

def get_nlp_service_keys():
  with open('gdrive/My Drive/passwords/nlp.keys') as f:
    return f.readlines()[0]

def get_newsapi_keys():
  with open('gdrive/My Drive/passwords/newsapi.keys') as f:
    return f.readlines()[0]

def getfmpkeys():
  with open('gdrive/My Drive/passwords/fmprep.keys') as f:
    return f.readlines()[0]




<h3> Fetching 10-q datset </h3>

In [ ]:
!gsutil cp   gs://mm_dataflow_bucket/inputs/quarterly_earnings_mixed_tickets_dataset.csv data

<h3> Filtering on BAC stock </h3>

In [ ]:
def setting_index(bac_df):
  bac_df = bac_df.set_index('cob_date')
  bac_df = bac_df.sort_index(ascending=True)
  return bac_df

def get_price_after_quarter(as_of_date, ticker):
  #print('Current as of date:{}={}'.format(as_of_date, type(as_of_date)))
  #print('ticker:{}'.format(ticker))
  if isinstance(as_of_date, date):
    dt = as_of_date
  else:
    dt = datetime.strptime(as_of_date, '%Y-%m-%d').date()
  try:
    price=  dr.get_data_yahoo(ticker, dt, dt)[['Close']].values.tolist()[0][0]
    #print('rice for {} is {}'.format(as_of_date, price))
    return price
  except Exception as e:
    print('Unable to find price for {} type){}, getting pre vday'.format(as_of_date, type(as_of_date)))
    prev_day = dt + BDay(1)
    return get_price_after_quarter(prev_day.date(), ticker)


def find_price(bac_df, ticker, field):
  bac_df['price'] = bac_df[field].apply(get_price_after_quarter, args=[ticker])
  return bac_df


def find_historical_price_iex(ticker, as_of_date):
  base_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/chart/1d/{date}?token={token}&chartByDay=true&chartCloseOnly=true'.format(symbol=ticker, date=as_of_date.strftime('%Y%m%d'),
                                                                                          token=get_iexapi_keys())
  print('getting from url{}'.format(base_url))
  return requests.get(base_url).json()




<h3> Using FinancialModellingPrep. We can still get 250 request per day for free..</h3>

<h3>First retrieving  all ticker and indicators </h3>


In [ ]:
def get_all_tickers():
  !gsutil cp gs://datascience-bucket-mm/all_sectors.csv  data
  df = pd.read_csv('data/all_sectors.csv')
  renamed = df.rename({'Unnamed: 0' : 'ticker'}, axis=1)

  # grouping by ticker
  print(renamed.groupby(['Sector']).count())
  return renamed




In [ ]:
# https://medium.com/swlh/teaching-a-machine-to-trade-stocks-like-warren-buffett-part-i-445849b208c6
def get_10q_statements(ticker):
  try:
    print('Getting tickers for {}'.format(ticker))
    base_url = 'https://financialmodelingprep.com/api/v3/key-metrics/{}?period=quarter&apikey=79d4f398184fb636fa32ac1f95ed67e6'.format(ticker.upper())
    print('Opening:{}'.format(base_url))
    res = requests.get(base_url).json()
    print('Respone has:{}'.format(len(res)))
    return pd.DataFrame(res).head(12)
  except Exception as e:
    print('Exception for {}:{}'.format(ticker, str(e)))

def get_10q_statement_iex(ticker, num_statements):
  base_url = 'https://cloud.iexapis.com/stable/time-series/REPORTED_FINANCIALS/{symbol}/10-Q?last={numstats}&token={token}'.format(symbol=ticker, 
                                                                                          numstats=num_statements,
                                                                                          token=get_iexapi_keys())
  print('Executing:{}'.format(base_url))
  data =  requests.get(base_url).json()
  return pd.DataFrame(data)
  

def get_tickers_for_sector(sector_name):
  print('Getting all tickers for {}'.format(sector_name))
  all_tickers = get_all_tickers()
  print('shape is:{}'.format(all_tickers.shape))
  sector_ticks = all_tickers[all_tickers['Sector'] == sector_name]
  return sector_ticks['ticker'].values.tolist()


def _create_10q_data_for_ticker(ticker):
  print('Fetching financial data....')
  find_df = get_10q_statements(ticker)
  if find_df is not None:
    with_price = find_price(find_df, ticker, 'date')
    return with_price


#stmnts = [get_10q_statements(t) for t in ['GE', 'WP']]
#all_dt = pd.concat(stmnts)
#print(all_dt.shape)
get_all_tickers()


<h3> Testing 10q statements on Amazon </h3>

In [ ]:
# Good enough for training, but not for latest. We'll see what we got from financialmodellingprep. Mayb we  shold spend July checking
def build_dataframe_for_ticker(ticker):
  try:
    res = get_10q_statements(ticker).dropna(axis=1)
    res['cob_date'] = res['date'].apply(lambda ds: datetime.strptime(ds, '%Y-%m-%d')).head(1)
    res = find_price(res, 'amzn', 'date')
    return res
  except Exception as  e:
    print('Cannot find statements for {}'.format(ticker))

In [ ]:
build_dataframe_for_ticker('AMZN')

In [ ]:
ticker_and_sectors = get_all_tickers()


In [ ]:
def get_tickers_for_sector(sector_name):
  print('Getting ticker for :{}'.format(sector_name))
  utilities_tickers = ticker_and_sectors[ticker_and_sectors['Sector'] == sector_name]
  return set(utilities_tickers.ticker.values.tolist()) 



In [ ]:
comm_svc = get_tickers_for_sector('Communication Services')
print('finding data for {}'.format(len(comm_svc)))
ut_dframes = dict((t, build_dataframe_for_ticker(t) ) for t in comm_svc)

In [ ]:
# Remember to filter out cob dates earlier than 2015
all_data = pd.concat([d for d in ut_dframes.values() if d is not None], axis=0)
print(all_data.shape)



In [ ]:
all_data.to_csv('all_data_utilities_df_1008.csv')
!gsutil cp  all_data_utilities_df_1008.csv gs://mm_dataflow_bucket/inputs/


In [ ]:
all_data.columns
all_data.dtypes

<h3> Now we shold find only relevant indicators. we have over 90 of them </h3>

In [ ]:
# Helper functions
def setting_index(df):
    """
    Returns a sorted datetime index
    """
    df['Quarter end'] = pd.to_datetime(df['date'])
    df.set_index("Quarter end", inplace=True)
    return df.sort_index(ascending=True)
  
def class_creation(df, thres=3):
    """
    Creates classes of:
    - buy(1)
    - hold(2)
    - sell(0)
    
    Threshold can be changed to fit whatever price percentage change is desired
    """
    if df['price'] >= thres :
        # Buys
        return 1
    
    elif df['price'] <= -thres :
        # Sells
        return 0
    
    else:
        # Holds
        return 2


In [ ]:
def prepare_data(stocks_df):
  for i in stocks_df.keys():
    print('Setting index..')
    stocks_df[i] = setting_index(stocks_df[i])
    
  # Replacing all "None" values with NaN
  for i in stocks_df.keys():
    print('Replacing noex.. for {}'.format(i))
    stocks_df[i].replace("None", 0, inplace=True)
      
  # Creating a new dictionary that contains the numerical values, then converting all values to numeric values
  num_df = {}
  print(' numeric..')
  for i in stocks_df.keys():
      num_df[i] = stocks_df[i].apply(lambda x: pd.to_numeric(x, errors='coerce'))
      
  # Replacing values with percent difference or change
  pcnt_df = {}
  for i in num_df.keys():
      pcnt_df[i] = num_df[i].pct_change(periods=1).apply(lambda x: x*100)
      
  # Replacing infinite values with NaN
  for i in pcnt_df.keys():
      print('Replacing infinte')
  
      pcnt_df[i] = pcnt_df[i].replace([np.inf, -np.inf], np.nan)
      
  # Creating a new DataFrame that contains the class 'Decision' determining if a quarterly reports improvement is a buy, hold, or sell.
  new_df = {}
  for i in pcnt_df.keys():
      print('Creating decision ')
  
      # Assigning the new DF
      new_df[i] = pcnt_df[i]
      
      # Creating the new column with the classes, shifted by -1 in order to know if the prices will increase/decrease in the next quarter.
      new_df[i]['Decision'] = new_df[i].apply(class_creation, axis=1).shift(-1)
      
  # Excluding the first and last rows
  for i in new_df.keys():
      new_df[i] = new_df[i][1:-1]
  return new_df
  

<h3>Build dictionary of tickers </h3>

In [ ]:
ticks =  all_data.symbol.values.tolist()
all_dict = dict((tick, all_data[all_data['symbol'] == tick]) for tick in ticks)

In [ ]:
all_data.columns
updated = prepare_data(all_dict)
type(updated)

In [ ]:
final_df = pd.concat([v for v in updated.values()])

In [ ]:
final_df.to_csv('all_data_utilities_df_1008_prepared.csv')
!gsutil cp  all_data_utilities_df_1008_prepared.csv gs://mm_dataflow_bucket/inputs/



In [ ]:
!gsutil cp gs://mm_dataflow_bucket/inputs/all_data_utilities_df_1008_prepared.csv .

In [ ]:
final_df = pd.read_csv('all_data_utilities_df_1008_prepared.csv')
final_df.shape

<h3> Visualizing data </h3>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import _pickle as pickle

# Separating each class into respective DataFrames
buy_df = final_df[final_df['Decision']==1].loc[:, final_df.columns != 'Decision'].reset_index(drop=True)
hold_df = final_df[final_df['Decision']==2].loc[:, final_df.columns != 'Decision'].reset_index(drop=True)
sell_df = final_df[final_df['Decision']==0].loc[:, final_df.columns != 'Decision'].reset_index(drop=True)

# Visualizing in matplotlib
plt.figure(figsize=(10,6))
plt.style.use('fivethirtyeight')

# Plotting the count of each DataFrame of each class
plt.bar("Buy", buy_df.shape[0])
plt.bar("Sell", sell_df.shape[0])
plt.bar("Hold", hold_df.shape[0])

plt.ylabel("# of Quarterly Reports")
plt.title('Count of Quarterly Reports')
plt.show()

<h3> Finding correlation in our data </h3>

In [ ]:
def CorrMtx(df, dropDuplicates = True):
    """
    Takes in a Correlation DF and excludes nonessential visuals.
    Creates a more visually pleasing correlation matrix
    """

    # Exclude duplicate correlations by masking uper right values
    if dropDuplicates:    
        mask = np.zeros_like(df, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True

    # Set background color / chart style
    sns.set_style(style = 'white')

    # Set up  matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Add diverging colormap from red to blue
    cmap = sns.diverging_palette(250, 10, as_cmap=True)

    # Draw correlation plot with or without duplicates
    if dropDuplicates:
        sns.heatmap(df, mask=mask, cmap=cmap, 
                square=True,
                linewidth=.5, cbar_kws={"shrink": .5}, ax=ax)
    else:
        sns.heatmap(df, cmap=cmap, 
                square=True,
                linewidth=.5, cbar_kws={"shrink": .5}, ax=ax)
        
        
# Correlation DF of all classes
corr = final_df.corr().iloc[[-1],:-1]

# Plotting the Correlation DF as a heatmap
plt.figure(figsize=(14,1))
sns.heatmap(corr, annot=False, linewidths=.1, cmap="coolwarm")
plt.xticks()
plt.yticks(rotation=0)
plt.show()

<h2>Feature Selection </h2>

<h4>Finding top 10 features </h4>

In [ ]:
def find_top_10(input_df):
  corr = input_df.corr().iloc[[-1],:-1]

  # Sorting our Correlation DF by their absolute values and selecting the top 10
  top10_corr = corr.transpose().apply(abs).sort_values(by='Decision', ascending=False)[:10]

  # Creating a new DF with the features from the top10_corr and joing the 'Decision' class labels
  top10_corr_df = input_df[top10_corr.index].join(final_df.Decision)

  # Pickling the DF for use in our Classification models
  with open("data/top10_corr_df_1008.pkl", "wb") as fp:
    pickle.dump(top10_corr_df, fp)
  !gsutil cp  data/top10_corr_df_1008.pkl gs://mm_dataflow_bucket/inputs/

  return top10_corr_df
  

In [ ]:
top10 = find_top_10(final_df)
top10.columns

<h4> Method 2. Using a Random Classifier </h4>

In [ ]:
# Importing the necessary libraries
from sklearn.ensemble import ExtraTreesClassifier

def find_top_10_via_classifier(input_df):
  # Instatiating the classifier
  forest = ExtraTreesClassifier(n_estimators=200)

  # Setting the corresponding variables for our classifier
  X = input_df.drop(['Decision'], 1)
  y = input_df.Decision

  # Fitting the classifier
  print('Fitting')
  forest.fit(X, y)

  # Determining the important features
  importances = forest.feature_importances_
  print('finding std')
  # The standard deviation among the trees for the important features
  std = np.std([i.feature_importances_ for i in forest.estimators_], axis=0)

  # Indexing and sorting the important features
  indices = np.argsort(importances)[::-1]

  # Assigning the top 10 features as a new DF
  top10_df = input_df[X.columns[indices][:10]].join(final_df.Decision)

  # Exporting the top 10 features DF
  with open("top10_df_classifier_1008.pkl", "wb") as fp:
      pickle.dump(top10_df, fp)
  return X, indices, importances

In [ ]:
cleaned = final_df.drop(columns=['Quarter end', 'date', 'cob_date']).fillna(0.0)
#print(cleaned.columns)
#print(cleaned.head(2).T)
#X.columns
#!gsutil cp  top10_df_classifier.pkl gs://mm_dataflow_bucket/inputs/
  

In [ ]:
no_idx = cleaned.reset_index()
X, indices, importances= find_top_10_via_classifier(no_idx)
!gsutil cp  top10_df_classifier_1008.pkl gs://mm_dataflow_bucket/inputs/


In [ ]:
!gsutil cp gs://mm_dataflow_bucket/inputs/top10_df_classifier.pkl .

In [ ]:
import pandas as pd

df = pd.read_pickle('top10_df_classifier.pkl')

In [ ]:
print(df.shape)
print(df.columns)

In [ ]:
# visualizing features
# Matplotlib style to use
plt.style.use('seaborn')

# Printing out the different features as a list
print("Feature Rankings:")

# Showing the top 10 features
for i in range(10):
    print(f"{i+1}. {X.columns[indices[i]]}: {importances[indices[i]]}")
    
# Plotting the top 10 features
plt.figure(figsize=(14,7))

plt.title("Feature Importances for the Original Dataset")
plt.bar(range(X.shape[1]), importances[indices], yerr=std[indices], align='center')

plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.xlim([-1, 11.5])
plt.show()